In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point



In [2]:
df_trips = pd.read_csv("data/trips.txt")
df_routes = pd.read_csv("data/routes.txt")
df_stops = pd.read_csv("data/stops.txt")
df_stop_times = pd.read_csv("data/stop_times.txt")
df_pathways = pd.read_csv("data/pathways.txt")

/var/folders/16/qc8wq8c93v3f89ksks8p_szm0000gn/T/ipykernel_24247/924667806.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trips = pd.read_csv("data/trips.txt")
/var/folders/16/qc8wq8c93v3f89ksks8p_szm0000gn/T/ipykernel_24247/924667806.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stop_times = pd.read_csv("data/stop_times.txt")


## Járatok

In [3]:
df_trips

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,8140,C0418310,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_5,Y701,1.0,1.0
1,8140,C0418311,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_6,Y702,1.0,1.0
2,8140,C0418312,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_7,Y701,1.0,1.0
3,8140,C0418313,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_8,Y702,1.0,1.0
4,8140,C0418314,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_9,Y701,1.0,1.0
...,...,...,...,...,...,...,...,...,...
294466,H5,H87191_4,61003,Batthyány tér,1,NaN,4,NaN,1.0
294467,H5,H87201_3,61003,Békásmegyer,0,NaN,3,NaN,1.0
294468,H5,H87291_4,61003,Batthyány tér,1,NaN,4,NaN,1.0
294469,H5,H87301_3,61004,Békásmegyer,0,NaN,3,NaN,1.0


## Melyik járat, melyik megállóban mikor áll meg?

In [4]:
df_stop_times


,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,C0418310,F04181,08:03:00,08:03:00,0,NaN,NaN,NaN,0.0
1,C0418310,F04526,08:06:00,08:06:00,1,NaN,NaN,NaN,160.0
2,C0418311,F04526,08:30:00,08:30:00,0,NaN,NaN,NaN,0.0
3,C0418311,F04181,08:33:00,08:33:00,1,NaN,NaN,NaN,160.0
4,C0418312,F04181,08:33:00,08:33:00,0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
6181322,H8739_4,9068197,04:04:00,04:04:00,7,NaN,NaN,NaN,6977.0
6181323,H8739_4,9050195,04:05:00,04:05:00,8,NaN,NaN,NaN,7581.0
6181324,H8739_4,9043193,04:07:00,04:07:00,9,NaN,NaN,NaN,8398.0
6181325,H8739_4,9019191,04:09:00,04:09:00,10,NaN,NaN,NaN,9750.0


In [5]:
df_stops


,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN
3,003105,Bécsi út / Vörösvári út,47.548817,19.029722,003105,NaN,NaN,NaN,2.0
4,003117,Hubay Jenő tér,47.562900,19.115709,003117,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
6148,MERGED_F01013,Astoria M,47.494145,19.060146,F01013,1.0,stop,NaN,2.0
6149,MERGED_F01031,Nyugati pályaudvar M (Podmaniczky utca),47.509276,19.058597,F01031,1.0,stop,NaN,1.0
6150,MERGED_F01401,Közvágóhíd H,47.467527,19.075886,F01401,1.0,stop,NaN,2.0
6151,MERGED_F02865,Nagy Lajos király útja / Czobor utca,47.522827,19.102034,F02865,1.0,stop,NaN,2.0


## Útvonalak

In [6]:
df_routes

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
0,BKK,0050,5,NaN,3,"Pasaréti tér / Rákospalota, Kossuth utca",009EE3,FFFFFF,19
1,BKK,0070,7,NaN,3,"Albertfalva vasútállomás / Újpalota, Nyírpalot...",009EE3,FFFFFF,23
2,BKK,0075,7E,NaN,3,"Blaha Lujza tér M / Újpalota, Nyírpalota út",009EE3,FFFFFF,24
3,BKK,0078,7G,NaN,3,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",009EE3,FFFFFF,25
4,BKK,0085,8E,NaN,3,"Kelenföld vasútállomás M / Újpalota, Nyírpalot...",009EE3,FFFFFF,27
...,...,...,...,...,...,...,...,...,...
381,HEV,H9,H9,NaN,109,Örs vezér tere / Csömör,ED6E86,FFFFFF,8
382,BKK,KT01,KT01,NaN,3,Örs vezér tere M+H / Kőbányai garázs,009EE3,FFFFFF,336
383,BKK,KTVB,KTVB,NaN,3,Óbudai autóbuszgarázs / Szentendre H,F9AB13,000000,338
384,BKK,MOZG,MOZG,NaN,3,Óbudai autóbuszgarázs / Széll Kálmán tér M,009EE3,FFFFFF,339


Példa: Keressük a 6-os villamost

In [7]:
df_routes[ df_routes["route_short_name"] == "6"]

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
252,BKK,3060,6,NaN,0,Széll Kálmán tér M / Móricz Zsigmond körtér M,FFD800,000000,21
385,BKK,VP06,6,NaN,3,Móricz Zsigmond körtér M / Széll Kálmán tér M,009EE3,FFFFFF,22


Példa: Keresünk útvonalszámokat:

In [8]:
df_routes[ df_routes["route_short_name"].isin(["6","7","9"])]

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
1,BKK,0070,7,NaN,3,"Albertfalva vasútállomás / Újpalota, Nyírpalot...",009EE3,FFFFFF,23
5,BKK,0090,9,NaN,3,"Óbuda, Bogdáni út / Kőbánya alsó vasútállomás",009EE3,FFFFFF,28
252,BKK,3060,6,NaN,0,Széll Kálmán tér M / Móricz Zsigmond körtér M,FFD800,000000,21
385,BKK,VP06,6,NaN,3,Móricz Zsigmond körtér M / Széll Kálmán tér M,009EE3,FFFFFF,22


Stop names from trips.

Keresünk egy routet-t és az első trip-et benne.
Ahhoz a triphez lekérdezzük a megállókat.



EA tripeket csoportosítom route_id szerint és kiveszem az első trip_id-t

In [9]:
df_trips.groupby(["route_id"])["trip_id"].transform("first").drop_duplicates()


0           C0418310
142        C11388145
318          C279651
1304         C564032
1314         C612092
             ...    
292146    H100027_37
292626    H150026_43
292828     H20002_11
293143      H30005_5
293748     H400019_1
Name: trip_id, Length: 590, dtype: object

Ez szerintem nagyon hasonlo, most group_by van route_id szerint, majd kiveszem az első route_id-t és trip_id-t és headsign-t

In [10]:
df_trips.groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})

,route_id,trip_id,trip_headsign
route_id,,,
50,50,D01676116,Pasaréti tér
70,70,D06143101,"Újpalota, Nyírpalota út"
75,75,D0172110,Blaha Lujza tér M
90,90,D05740102,"Óbuda, Bogdáni út"
110,110,D0617810,Batthyány tér M H
...,...,...,...
H6,H6,H20002_11,Ráckeve
H7,H7,H30005_5,Csepel
H8,H8,H100027_37,Gödöllő


Ez most csak játék a 4-6 os villamos első induását / járatát keresem:

In [11]:
df_trips[ df_trips["route_id"].isin(["3060","3040"]) ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})

,route_id,trip_id,trip_headsign
route_id,,,
3040,3040,D015151066,Széll Kálmán tér M
3060,3060,D015151403,Móricz Zs. körtér M


# Melyik megállókban áll meg a 4-6-os ?

### Mikor indul reggel a 4-6-os

A 3060-as járat első indulását keresem.
A groupby és agg funckiók miatt fontos

In [12]:
df_trips[ df_trips["route_id"].isin(["3060"]) ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"]

route_id
3060    D015151403
Name: trip_id, dtype: object

### Melyik megállókat érinti a D015151403 járat?


In [13]:
df_stop_times[ df_stop_times["trip_id"] == "D015151066"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
1155147,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0
1155148,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0
1155149,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0
1155150,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0
1155151,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0
1155152,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0
1155153,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0
1155154,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0
1155155,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0
1155156,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0


### Most tegyük mellé a neveket is!

In [14]:
pd.merge( df_stop_times[ df_stop_times["trip_id"] == "D015151066"], df_stops, on="stop_id")

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0,Újbuda-központ M,47.474042,19.046862,F01998,NaN,NaN,NaN,1.0
1,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0,Budafoki út / Szerémi sor,47.474114,19.053747,F01992,NaN,NaN,NaN,1.0
2,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0,"Petőfi híd, budai hídfő",47.476956,19.059572,F02225,NaN,NaN,NaN,1.0
3,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0,Boráros tér H,47.480400,19.066697,F01374,NaN,NaN,NaN,2.0
4,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0,Mester utca / Ferenc körút,47.482775,19.068848,F01380,NaN,NaN,NaN,1.0
5,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0,Corvin-negyed M,47.485500,19.069924,F01191,NaN,NaN,NaN,1.0
6,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0,Harminckettesek tere,47.490357,19.070841,F01199,NaN,NaN,NaN,1.0
7,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0,Rákóczi tér M,47.492866,19.071187,F01200,NaN,NaN,NaN,1.0
8,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0,Blaha Lujza tér M,47.496770,19.070725,F01168,NaN,NaN,NaN,1.0
9,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0,Wesselényi utca / Erzsébet körút,47.500350,19.068934,F01111,NaN,NaN,NaN,1.0


# Kódrészlet ami egy vonal megállóit keresi meg
Most egy kis kódrészlet keresi nekem az adott vonal megállóit

In [15]:
#route_id = "3060"
#route_id="3040"
route_id="0090"
elso_trip = df_trips[ df_trips["route_id"] == route_id ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"].iloc[0]
a=pd.merge( df_stop_times[ df_stop_times["trip_id"] == elso_trip], df_stops, on="stop_id")
a["route_id"]=route_id
a[["route_id","trip_id","stop_id","stop_name","arrival_time","departure_time","stop_lat","stop_lon"]]

,route_id,trip_id,stop_id,stop_name,arrival_time,departure_time,stop_lat,stop_lon
0,0090,D0764412,F01636,Kőbánya alsó vasútállomás,05:12:00,05:12:00,47.483652,19.127839
1,0090,D0764412,F01653,Szent László tér,05:13:00,05:13:00,47.485925,19.131093
2,0090,D0764412,F01641,Liget tér,05:15:00,05:15:00,47.482795,19.130358
3,0090,D0764412,F01571,Kőbánya alsó vasútállomás (Mázsa tér),05:17:00,05:17:00,47.482879,19.125715
4,0090,D0764412,F01563,Egészségház,05:18:00,05:18:00,47.484126,19.116668
5,0090,D0764412,F01733,Eiffel Műhelyház,05:19:00,05:19:00,47.485136,19.112914
6,0090,D0764412,F01267,Kőbányai út / Könyves Kálmán körút,05:21:00,05:21:00,47.486740,19.106971
7,0090,D0764412,008036,Kőbányai út 31.,05:21:00,05:21:00,47.487461,19.104252
8,0090,D0764412,F01187,Orczy tér,05:23:00,05:24:00,47.489470,19.090495
9,0090,D0764412,F01223,Kálvária tér,05:25:00,05:25:00,47.488890,19.085146


## Melyik járatok állnak meg ebben a megállóban ?

In [16]:
df_stop_times[ df_stop_times["stop_id"] == "048506"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
566546,C96610106,048506,07:27:00,07:27:00,9,Zugló vasútállomás,NaN,NaN,3713.0
566598,C96610111,048506,07:37:00,07:37:00,9,Zugló vasútállomás,NaN,NaN,3713.0
566648,C96610116,048506,07:44:00,07:44:00,9,Zugló vasútállomás,NaN,NaN,3713.0
566700,C96610121,048506,07:52:00,07:52:00,9,Zugló vasútállomás,NaN,NaN,3713.0
566725,C966101221,048506,16:35:00,16:35:00,9,Zugló vasútállomás,NaN,NaN,3713.0
...,...,...,...,...,...,...,...,...,...
6040371,D0875684,048506,08:34:00,08:34:00,17,NaN,NaN,NaN,8094.0
6040436,D0875687,048506,08:42:00,08:42:00,17,NaN,NaN,NaN,8094.0
6040501,D087569,048506,05:25:00,05:25:00,17,NaN,NaN,NaN,8094.0
6040533,D0875690,048506,08:51:00,08:51:00,17,NaN,NaN,NaN,8094.0


Látszik, hogy jó sok indulás van ebből a megállóból

## Melyik útvonalak állnak meg ebben a megállóban?


In [17]:
stop_id = "048506"
#stop_id = "F01029"
parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

In [18]:
if parent_stop_id is not np.nan:
        stop_id = parent_stop_id
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name

'Szent István Bazilika'

In [19]:
df_megallo_jaratok = pd.merge( df_stop_times[ df_stop_times["stop_id"] == stop_id ], df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "stop_headsign", "stop_headsign","stop_sequence"]]


,stop_id,stop_name,route_id,arrival_time,stop_headsign,stop_headsign,stop_sequence
549,048506,Szent István Bazilika,9310,00:20:00,Nyugati pályaudvar M ► Zöldmál,Nyugati pályaudvar M ► Zöldmál,2
567,048506,Szent István Bazilika,9310,00:20:00,Nyugati pályaudvar M ► Zöldmál,Nyugati pályaudvar M ► Zöldmál,2
1840,048506,Szent István Bazilika,9501,00:30:00,"Rákospalota, Kossuth utca","Rákospalota, Kossuth utca",12
1875,048506,Szent István Bazilika,9140,00:39:00,"Káposztásmegyer, Mogyoródi-patak","Káposztásmegyer, Mogyoródi-patak",30
572,048506,Szent István Bazilika,9500,01:09:00,"Rákospalota, Székely Elek út","Rákospalota, Székely Elek út",42
1814,048506,Szent István Bazilika,9500,01:09:00,"Rákospalota, Székely Elek út","Rákospalota, Székely Elek út",42
1808,048506,Szent István Bazilika,9141,01:16:00,NaN,NaN,12
573,048506,Szent István Bazilika,9501,01:30:00,"Rákospalota, Kossuth utca","Rákospalota, Kossuth utca",42
234,048506,Szent István Bazilika,4720,04:48:00,Zugló vasútállomás,Zugló vasútállomás,9
719,048506,Szent István Bazilika,90,05:01:00,"Óbuda, Bogdáni út","Óbuda, Bogdáni út",17


In [20]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time.str.startswith("09")'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "stop_headsign", "stop_headsign","stop_sequence"]]


,stop_id,stop_name,route_id,arrival_time,stop_headsign,stop_headsign,stop_sequence
108,048506,Szent István Bazilika,0090,09:00:00,NaN,NaN,17
42,048506,Szent István Bazilika,4720,09:00:00,Zugló vasútállomás,Zugló vasútállomás,9
36,048506,Szent István Bazilika,90,09:05:00,"Óbuda, Bogdáni út","Óbuda, Bogdáni út",17


In [21]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time > "090300" and arrival_time'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "stop_headsign", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,stop_headsign,stop_headsign,stop_sequence
1592,048506,Szent István Bazilika,0090,09:00:00,NaN,NaN,17
651,048506,Szent István Bazilika,4720,09:00:00,Zugló vasútállomás,Zugló vasútállomás,9
528,048506,Szent István Bazilika,90,09:05:00,"Óbuda, Bogdáni út","Óbuda, Bogdáni út",17
1342,048506,Szent István Bazilika,F4720,10:32:00,Zugló vasútállomás,Zugló vasútállomás,9
1341,048506,Szent István Bazilika,9500,24:09:00,"Rákospalota, Székely Elek út","Rákospalota, Székely Elek út",12
1338,048506,Szent István Bazilika,9140,24:19:00,"Káposztásmegyer, Mogyoródi-patak","Káposztásmegyer, Mogyoródi-patak",30


In [22]:
df_megallo_jaratok2 = pd.merge( df_megallo_jaratok , df_routes, left_on="route_id", right_on="route_id" )
df_megallo_jaratok2[["stop_id","stop_name","route_id","arrival_time", "route_short_name", "stop_headsign", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,route_short_name,stop_headsign,stop_headsign,stop_sequence
0,048506,Szent István Bazilika,0090,09:00:00,9,NaN,NaN,17
1,048506,Szent István Bazilika,4720,09:00:00,72,Zugló vasútállomás,Zugló vasútállomás,9
2,048506,Szent István Bazilika,F4720,10:32:00,72❆,Zugló vasútállomás,Zugló vasútállomás,9
3,048506,Szent István Bazilika,9500,24:09:00,950,"Rákospalota, Székely Elek út","Rákospalota, Székely Elek út",12
4,048506,Szent István Bazilika,9140,24:19:00,914,"Káposztásmegyer, Mogyoródi-patak","Káposztásmegyer, Mogyoródi-patak",30


Ez sajnos nagyon nem OK. Itt több más járatot is jó lenne látni.

## Mi a nyugati főmegállója?

Hát vagy ez vagy nincs kötlve a táblázat rendesen

In [23]:
stop_id = "F00933"
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name


'Nyugati pályaudvar M'

In [24]:
stop_id = "F01029"
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name


'Nyugati pályaudvar M'

In [25]:
stop_id = "F00933"
stop_id = "F01029"

parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

Melyek azok az állomások amiknek van főmegállója?

In [26]:
df_stops[  df_stops["parent_station"] != np.nan ]
df_stops[  df_stops["parent_station"] != "NaN" ]
df_stops[  df_stops["parent_station"].notna() ]


,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
200,008359,Óbudai autóbuszgarázs,47.567996,19.026289,F00678,NaN,merged,MERGED_F00678,1.0
201,008365,Bécsi út / Vörösvári út,47.548489,19.028994,F00568,NaN,merged,MERGED_F00568,1.0
262,008545,Puskás Ferenc Stadion M,47.500105,19.107013,008545,NaN,NaN,008875,1.0
434,009085,Budaörs vasútállomás,47.448166,18.964870,009085,NaN,NaN,008690,1.0
653,009630,"Budakalász, Auchan áruház",47.611510,19.062341,009630,NaN,NaN,008822,1.0
...,...,...,...,...,...,...,...,...,...
6136,LMXKVC,Kálvin tér [C],47.489727,19.062364,LMXKVC,2.0,NaN,CS056227,2.0
6137,LMXKVD,Kálvin tér [D],47.489226,19.061814,LMXKVD,2.0,NaN,CS056227,2.0
6138,LMXKVE,Kálvin tér [E],47.489559,19.061525,LMXKVE,2.0,NaN,CS056227,2.0
6139,LMXKVF,Kálvin tér [F],47.489696,19.061327,LMXKVF,2.0,NaN,CS056227,2.0


In [27]:
df_stops[ df_stops["parent_station"] == "F01029"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding


In [28]:
df_stops[ df_stops["parent_station"] == "008875"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
262,008545,Puskás Ferenc Stadion M,47.500105,19.107013,008545,NaN,NaN,008875,1.0


In [29]:
df_stops[ df_stops["parent_station"] == "CS056227"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
905,056227,Kálvin tér,47.488903,19.061022,056227,NaN,NaN,CS056227,1.0
906,056228,Kálvin tér,47.489245,19.061938,056228,NaN,NaN,CS056227,1.0
2768,F01289,Kálvin tér,47.490170,19.060640,F01289,NaN,NaN,CS056227,1.0
2769,F01290,Kálvin tér,47.489490,19.061920,F01290,NaN,NaN,CS056227,1.0
6017,LM3KV1,Kálvin tér,47.489693,19.061707,LM3KV1,3.0,NaN,CS056227,1.0
6095,LM4KV1,Kálvin tér,47.489048,19.061394,LM4KV1,3.0,NaN,CS056227,2.0
6133,LMXKV1,Kálvin tér (lift),47.489010,19.061593,LMXKV1,2.0,NaN,CS056227,1.0
6134,LMXKVA,Kálvin tér [A],47.489987,19.061298,LMXKVA,2.0,NaN,CS056227,2.0
6135,LMXKVB,Kálvin tér [B],47.489919,19.062093,LMXKVB,2.0,NaN,CS056227,2.0
6136,LMXKVC,Kálvin tér [C],47.489727,19.062364,LMXKVC,2.0,NaN,CS056227,2.0


# KÖZELI ÁTSZÁLLÁSI PONTOK

Ez úgy tűnik földrajzi koordináták alapján kell megkeresni.

In [30]:

# Create a list of Point objects
points = [Point(xy) for xy in zip(df_stops['stop_lon'], df_stops['stop_lat'])]

# Create a GeoSeries from the list of points with the correct CRS
gs = gpd.GeoSeries(points, crs='EPSG:4326')

gdf_stops = gpd.GeoDataFrame(df_stops, geometry=gs, crs="EPSG:4326")
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (19.1357 47.50037)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (19.12789 47.48314)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (19.10341 47.50037)
3,003105,Bécsi út / Vörösvári út,47.548817,19.029722,003105,NaN,NaN,NaN,2.0,POINT (19.02972 47.54882)
4,003117,Hubay Jenő tér,47.562900,19.115709,003117,NaN,NaN,NaN,1.0,POINT (19.11571 47.5629)


Az EPSG:4326 nem jó távolságmérésre, mert nem tudom miért.
De a 3035 az jó, Németországhoz kötött valami. Ezért átmegyek arra:

In [31]:
gdf_stops.to_crs("epsg:3035",inplace=True)
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (5007781.301 2752100.781)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (5007426.257 2750125.866)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (5005367.638 2751801.574)
3,003105,Bécsi út / Vörösvári út,47.548817,19.029722,003105,NaN,NaN,NaN,2.0,POINT (4999223.77 2756473.426)
4,003117,Hubay Jenő tér,47.562900,19.115709,003117,NaN,NaN,NaN,1.0,POINT (5005458.179 2758821.276)


Most a Nyugatihoz viszonyítva fogunk közeli megállókat keresni

In [32]:
stop_id = "F01029" # Nyugati
gdf_nyugati = gdf_stops [ gdf_stops["stop_id"] == stop_id ]
gdf_megallo = gdf_nyugati.to_crs(epsg=3035)
gdf_megallo

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
2537,F01029,Nyugati pályaudvar M,47.509806,19.057151,F01029,NaN,NaN,NaN,1.0,POINT (5001786.02 2752416.904)


In [33]:
gdf_nearest_stops = gdf_megallo.sjoin_nearest(gdf_stops,distance_col="distance", how="right", exclusive = True, max_distance = 500)
gdf_nearest_stops.columns
#gdf_nearest_stops[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]


Index(['index_left', 'stop_id_left', 'stop_name_left', 'stop_lat_left',
       'stop_lon_left', 'stop_code_left', 'location_type_left',
       'location_sub_type_left', 'parent_station_left',
       'wheelchair_boarding_left', 'stop_id_right', 'stop_name_right',
       'stop_lat_right', 'stop_lon_right', 'stop_code_right',
       'location_type_right', 'location_sub_type_right',
       'parent_station_right', 'wheelchair_boarding_right', 'geometry',
       'distance'],
      dtype='object')

In [34]:
gdf_nearest_stops.nsmallest(40,'distance')[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]

,stop_id_left,stop_name_left,stop_lat_left,stop_lon_left,stop_id_right,stop_name_right,stop_lat_right,stop_lon_right,distance
2461,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00935,Nyugati pályaudvar M,47.509922,19.056658,39.253332
6061,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYI,Nyugati pályaudvar [I],47.509870,19.056404,56.669957
2460,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00934,Nyugati pályaudvar M,47.509896,19.056394,57.835671
6057,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYE,Nyugati pályaudvar [E],47.510447,19.056426,89.626772
6060,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYH,Nyugati pályaudvar [H],47.510233,19.056103,91.944253
2459,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00933,Nyugati pályaudvar M,47.510300,19.056167,92.071854
6059,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYG,Nyugati pályaudvar [G],47.510326,19.056196,92.107868
6052,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NY3,Nyugati pályaudvar (lift » M3),47.510040,19.055784,106.075445
127,F01029,Nyugati pályaudvar M,47.509806,19.057151,008137,Nyugati pályaudvar,47.510571,19.056072,117.418620
6058,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYF,Nyugati pályaudvar [F],47.510703,19.056257,120.137919


In [35]:
stop_id = "F01029"
df_pathways


,pathway_id,pathway_mode,is_bidirectional,from_stop_id,to_stop_id,traversal_time
0,LM4KF1-LM4KFA,1,1,LM4KF1,LM4KFA,90
1,LM4KF1-LM4KFB,2,1,LM4KF1,LM4KFB,180
2,LM3UK2-F00913,5,1,LM3UK2,F00913,90
3,LM4KF1-LM4KFC,2,1,LM4KF1,LM4KFC,210
4,LM4KF1-LM4KFD,2,1,LM4KF1,LM4KFD,180
...,...,...,...,...,...,...
488,LM3NT1-F01252-WC,5,1,LM3NT1,F01252,60
489,LM3AHF-LM3AH1,2,1,LM3AHF,LM3AH1,180
490,LM3EU1-F01494,5,1,LM3EU1,F01494,90
491,L008165-F00898,2,1,L008165,F00898,30


In [36]:
stop_list = df_pathways [ df_pathways ["from_stop_id"] == stop_id]
stop_list

,pathway_id,pathway_mode,is_bidirectional,from_stop_id,to_stop_id,traversal_time
